In [257]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
import pandas

In [258]:
headers = {
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0',
}
first_pg_sj='https://www.superjob.ru/vakansii/shveya.html?geo%5Bt%5D%5B0%5D=4'

In [196]:
response = requests.get(first_pg_sj, headers=headers) 
soup = bs(response.content, 'html.parser')
tag_main_div=soup.find_all('div', {'class': '_2J-3z _3B5DQ'})
tag_a_name=tag_main_div[0].find('a').text

In [259]:
def total_parsing_sj(first_pg, headers, result=[]):

    response = requests.get(first_pg, headers=headers) 
    
    if response.status_code != 200:
        print('Парсинг завершен')
        return result
    soup = bs(response.content, 'html.parser')
    tag_main_div=soup.find_all('div', {'class': '_2J-3z _3B5DQ'})
    for vacancy in tag_main_div:
        result.append(vac_parsing(vacancy))
    
    link_next_page = soup.find('a', {'class': '_1IHWd _6Nb0L _37aW8 _1Sycm f-test-button-dalshe f-test-link-Dalshe'})
    if link_next_page:
        link_next_page =  'https://www.superjob.ru/' + link_next_page['href']
        
    else:
        print('Парсинг завершен')
        return result
    result=total_parsing_sj(link_next_page, headers,result)                        
    return result
    

In [260]:
hh_vcncy_list=[]
def salary_split(tag_span_salary, vac_min_salary=None, vac_max_salary=None, currency=None):
    sleep(0.1)
    tag_span_salary=tag_span_salary.split('\xa0')
    
    if tag_span_salary[2]=='—':
        vac_max_salary=int(tag_span_salary[3]+tag_span_salary[4])
        vac_min_salary=int(tag_span_salary[0]+tag_span_salary[1])
        
    elif  len(tag_span_salary)==3:
        if (tag_span_salary[0]=='от' or tag_span_salary[0]=='до')and tag_span_salary[2]=='руб.':
            vac_max_salary=int(tag_span_salary[1])
            vac_min_salary=int(tag_span_salary[1])
            
    elif tag_span_salary[0]=='до':        
        vac_max_salary=(tag_span_salary[1] + tag_span_salary[2])
        
    elif tag_span_salary[0]=='от':
        vac_min_salary=(tag_span_salary[1] + tag_span_salary[2])
        
    currency=tag_span_salary[-1]
        
    
    return vac_min_salary, vac_max_salary, currency
def vac_parsing(tag_main_div):
    sleep(0.1)
    if tag_main_div.find('a'):
        tag_a_name=tag_main_div.find('a').text
        tag_a_vcnc_link='https://www.superjob.ru'+tag_main_div.find('a')['href']
    else: 
        return
    

    if tag_main_div.find('span', {'class', '_2eYAG _1nqY_ _249GZ _1jb_5 _1dIgi'}):
        tag_span_salary=tag_main_div.find('span', {'class', '_2eYAG _1nqY_ _249GZ _1jb_5 _1dIgi'}).text

        if tag_span_salary!='По договорённости':
              vac_min_salary, vac_max_salary, currency=salary_split(tag_span_salary)
        else:
            vac_min_salary, vac_max_salary, currency=None,None,None
            
    else:
        return
        
    
    return {
        'name': tag_a_name,
        'min salary': vac_min_salary,
        'max salary': vac_max_salary,
        'currency': currency,
        'vac_link': tag_a_vcnc_link,
        'site':'https://www.superjob.ru'
        }

In [261]:
result=total_parsing_sj(first_pg_sj,headers)

Парсинг завершен


In [265]:
fine=list(filter(None, result))

In [266]:
data_frame_sj = pandas.DataFrame(fine) 

data_frame_sj.to_csv('sj.csv') 